# Présentation de la base de données & Tests de stationnarité


Nous avons exporté les données de l'OCDE dans une base de données que nous avons reformatée, afin d'avoir une indexation multiple au niveau des colonnes : un premier niveau pour les pays et ensuite pour chaque pays les différents indicateurs choisis. Nous avons aussi ajouté des indicateurs concernant les inégalités, à savoir le top 50% du revenu et le top 10% du revenu. Néanmoins nous nous posons quelques questions concernant ces indicateurrs. 1) Le coefficient de Gini est disponible mais le calcul n'est fait que par année (car il nous semble qu'il varie peu). Faut-il alors qu'on le calcule à la main pour chaque trimestre, l'utiliser tel quel (par annnée), ou ne pas l'utiliser du tout? 2) Est-il important de savoir si la part des revenus est bruts ou non, dans le mesure où l'on cherche à évaluer l'impact des politiques fiscales de Trump?

# Partie 1 : Création d'une base de données synthétique pour l'OCDE

In [74]:
import pandas as pd

In [75]:
bottom_50 = pd.read_csv("bottom_50_income.csv",
                       header = 1,
                       sep=';').sort_index(axis = 1)

In [76]:
top_10 =pd.read_csv(r"top_10_income.csv",
                   header = 1,
                   sep=';').sort_index(axis = 1)

In [11]:
#Note à nous mêmes : le ocde_df est maintenant à jour avec toutes les données (inégalités etc), donc 
#pas besoin refaire tourner ce code. il suffit de faire la commande table = pd.read_csv('ocde_df.csv',header = [1,2]) pour 
#avoir la table avec le double index (il faut simplement possiblement changer le path)

pays_ocde = {'AT' :'Austria','AU':'Australia','BE':'Belgium','CA':'Canada','CH':'Switzerland','DE':'Germany','DK':'Denmark',
             'ES':'Spain','FI':'Finland','FR':'France','GB':'United Kingdom','GR':'Greece','IE':'Ireland','IT':'Italy',
             'JP':'Japan','LU':'Luxembourg','NL':'Netherlands','NO':'Norway','NZ':'New Zealand','PT':'Portugal','SE':'Sweden',
             'TR':'Turkey','US':'United-States'}

df_ocde = pd.read_csv(r"ocde_df.csv")

df_ocde.rename(columns={'Unnamed: 0':'Variables'}, inplace=True)

ind_tuple = list(zip(df_ocde['Pays'], df_ocde['Variables']))
new_index = pd.MultiIndex.from_tuples(ind_tuple, names=["Pays", "Variables"])

# Mise en place du système de double indice

tocde = df_ocde.T.copy()
tocde.columns = new_index
tocde.drop(['Variables', 'Pays'], inplace=True)

# tocde = tocde.drop(['2020-Q1','2020-Q2','2020-Q3','2020-Q4','2021-Q1'])
# On renomme les colonnes de bottom_50 et top_10

bottom_50.columns = ['Variables', 'Year'] + list(pays_ocde[d] for d in sorted(pays_ocde.keys(), key=str))
top_10.columns = ['Variables', 'Year'] + list(pays_ocde[d] for d in sorted(pays_ocde.keys(), key=str))

# On prépare bottom_50 pour la fusion

bottom_50 = bottom_50.drop('Variables', axis=1)
for i in range (1,3) : 
    bottom_50 = pd.concat([bottom_50,bottom_50]) 
    
annee = []
for j in range (1,5):
    for i in range (0,29):
        annee.append(str(1991+i)+'-Q'+str(j))
bottom_50['Year'] = annee

# Mise en place double index pour bottom_50 

bottom_50 = bottom_50.set_index('Year')

bottom_50['Variables'] = 'income p0p50'

ind_tuple = list(zip(bottom_50.columns, bottom_50['Variables']))
new_index = pd.MultiIndex.from_tuples(ind_tuple, names=["Pays", "Variables"])
bottom_50.columns = new_index

bottom_50 = bottom_50.drop(('Variables','income p0p50'), axis = 1)

tocde = tocde.merge(right=bottom_50, how='outer',
                    left_index=True,
                    right_index=True).sort_index(axis = 0).sort_index(axis = 1)

# On prépare top_10 pour la fusion

top_10 = top_10.drop('Variables', axis = 1)
for i in range (1,3) : 
    top_10 = pd.concat([top_10,top_10])    
    
annee = []
for j in range (1,5):
    for i in range (0,29):
        annee.append(str(1991+i)+'-Q'+str(j))
top_10['Year'] = annee

# Mise en place double index pour top_10 :
top_10 = top_10.set_index('Year')

top_10['Variables'] = 'income p90p100'

ind_tuple = list(zip(top_10.columns, top_10['Variables']))

new_index = pd.MultiIndex.from_tuples(ind_tuple, names=["Pays", "Variables"])

top_10.columns = new_index

top_10 = top_10.drop(('Variables','income p90p100'), axis=1)

tocde = tocde.merge(right = top_10,
                    how='outer',
                    left_index=True,
                    right_index=True).sort_index(axis = 0).sort_index(axis = 1)

qs = tocde.index.str.replace(r'(Q\d) (\d+)', r'\2-\1')

tocde['date'] = pd.PeriodIndex(qs, freq='Q').to_timestamp()


tocde = tocde.reindex(index=tocde['date'])
tocde = tocde.drop("date", axis=1)
#tocde.drop(list(pd.date_range('20200101', periods=5, freq='QS-JAN')), inplace=True)



KeyError: 'Variables'

*QUESTION* : Que faut-il faire des valeurs manquantes indiquée par NaN? Les remplacer par 0 fausserait les résultalts

Nous allons désormais effectuer des tests de stationnarité sur les différentes variables. Nous appliquons deux tests  : KPSS qui a pour hypothèse nulle H0:"La série est stationnaire" et le test AFD dont l'hypothèse nulle est H0 : "La série est non stationnaire". NB : nous n'avons pas désaisonnalisé les séries avant d'effectuer les tests car nous ne savions pas si cela éait nécessaire.

# Partie 2 : Tests de stationnarité sur différentes séries temporelles

Dans un premier temps nous nous intéressons au PIB des Etats-Unis, puis aux autres séries issues du même pays. Nous nous intéresserons par la suite à quelques données issues d'autres pays de l'OCDE. 

Nous effectuons à chaque fois et alternativement deux tests : **KPSS** et **ADF** (Augmented Dickey-Fuller)

In [78]:
from statsmodels.tsa.stattools import kpss
from statsmodels.tsa.stattools import adfuller

In [79]:
def kpss_test(series, **kw):    
    statistic, p_value, n_lags, critical_values = kpss(series, **kw)
    
    # Format Output
    print(f'KPSS Statistic: {statistic}')
    print(f'p-value: {p_value}')
    print(f'num lags: {n_lags}')
    print('Critial Values:')
    for key, value in critical_values.items():
        print(f'   {key} : {value}')
    print(f'Result: The series is {"not " if p_value < 0.05 else ""}stationary')
    
def adf_test(series, **kw):    
    statistic, p_value, n_lags, nobs, critical_values, icbest = adfuller(series, **kw)
    # Format Output
    print(f'ADF Statistic: {statistic}')
    print(f'p-value: {p_value}')
    print(f'num lags: {n_lags}')
    print('Critial Values:')
    for key, value in critical_values.items():
        print(f'   {key} : {value}')
    print(f'Result: The series is {"not " if p_value > 0.05 else ""}stationary')

In [80]:
serie1=tocde['United-States']['PIB']
kpss_test(serie1)

KPSS Statistic: 0.9556147530640225
p-value: 0.01
num lags: 13
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
Result: The series is not stationary


C:\Users\SURFACE\Anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1850: FutureWarning: The behavior of using nlags=None will change in release 0.13.Currently nlags=None is the same as nlags="legacy", and so a sample-size lag length is used. After the next release, the default will change to be the same as nlags="auto" which uses an automatic lag length selection method. To silence this warning, either use "auto" or "legacy"
  warnings.warn(msg, FutureWarning)
C:\Users\SURFACE\Anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1882: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  warn_msg.format(direction="smaller"), InterpolationWarning


La série n'est pas stationnaire au seuil de 5%, nous allons donc appliquer le test à la série intégrée à l'ordre 1.

In [81]:
serie2 = serie1 - serie1.shift()
serie2.dropna(inplace=True)
kpss_test(serie2)

KPSS Statistic: 0.08667738001103942
p-value: 0.1
num lags: 13
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
Result: The series is stationary


C:\Users\SURFACE\Anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1886: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warn_msg.format(direction="greater"), InterpolationWarning


In [82]:
import matplotlib.pyplot as plt 

fig = plt.figure(figsize=(12, 7))
ax = fig.add_subplot(111)

serie2.plot(linewidth=3)

plt.title("Evolution stationnarisée du PIB Américain")
plt.show() 

<Figure size 1200x700 with 1 Axes>

La série intégrée à l'ordre 1 est stationnaire selon le test KPSS. Maintenant, effectuons le test ADF.

In [83]:
adf_test(serie1)
adf_test(serie2)

ADF Statistic: -0.030660868751717637
p-value: 0.955921802423386
num lags: 2
Critial Values:
   1% : -3.487517288664615
   5% : -2.8865777180380032
   10% : -2.5801239192052012
Result: The series is not stationary
ADF Statistic: -5.015879029087155
p-value: 2.06420289297421e-05
num lags: 1
Critial Values:
   1% : -3.487517288664615
   5% : -2.8865777180380032
   10% : -2.5801239192052012
Result: The series is stationary


Comme pour le test KPSS, la série intégrée à l'ordre 1 est stationnaire.
Maintenant effectuons les tests sur le chômage

In [84]:
serie1=tocde['United-States']['Chomage']

In [85]:
kpss_test(serie1)

KPSS Statistic: 0.09403467015765274
p-value: 0.1
num lags: 13
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
Result: The series is stationary


C:\Users\SURFACE\Anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1850: FutureWarning: The behavior of using nlags=None will change in release 0.13.Currently nlags=None is the same as nlags="legacy", and so a sample-size lag length is used. After the next release, the default will change to be the same as nlags="auto" which uses an automatic lag length selection method. To silence this warning, either use "auto" or "legacy"
  warnings.warn(msg, FutureWarning)
C:\Users\SURFACE\Anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1886: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warn_msg.format(direction="greater"), InterpolationWarning


In [86]:
adf_test(serie1)

ADF Statistic: -2.257662581670266
p-value: 0.18598624704216382
num lags: 9
Critial Values:
   1% : -3.4912451337340342
   5% : -2.8881954545454547
   10% : -2.5809876033057852
Result: The series is not stationary


In [87]:
serie2 = serie1 - serie1.shift()
serie2.dropna(inplace=True)

In [88]:
adf_test(serie2)

ADF Statistic: -4.072134155061354
p-value: 0.00107650059115743
num lags: 7
Critial Values:
   1% : -3.490683082754047
   5% : -2.8879516565798817
   10% : -2.5808574442009578
Result: The series is stationary


La série du chomage des Etats-Unis est stationnaire d'après le test KPSS, nous n'avons pas besoin de l'intégrer à l'ordre 1. Cependant elle n'est pas stationnaire selon le test AFD. Elle est stationnaire selon le test AFD intégrée à l'ordre 1. Maintenant intéressons nous à l'emploi.

In [89]:
serie1=tocde['United-States']['Emplois']

In [90]:
kpss_test(serie1)

KPSS Statistic: 0.562377817476475
p-value: 0.027617608676469594
num lags: 13
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
Result: The series is not stationary


C:\Users\SURFACE\Anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1850: FutureWarning: The behavior of using nlags=None will change in release 0.13.Currently nlags=None is the same as nlags="legacy", and so a sample-size lag length is used. After the next release, the default will change to be the same as nlags="auto" which uses an automatic lag length selection method. To silence this warning, either use "auto" or "legacy"
  warnings.warn(msg, FutureWarning)


In [91]:
serie2 = serie1 - serie1.shift()
serie2.dropna(inplace=True)
kpss_test(serie2)

KPSS Statistic: 0.09286187886977079
p-value: 0.1
num lags: 13
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
Result: The series is stationary


C:\Users\SURFACE\Anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1886: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warn_msg.format(direction="greater"), InterpolationWarning


In [92]:
adf_test(serie1)

ADF Statistic: -1.76780601924589
p-value: 0.39655813857384226
num lags: 3
Critial Values:
   1% : -3.4880216384691867
   5% : -2.8867966864160075
   10% : -2.5802408234244947
Result: The series is not stationary


In [93]:
adf_test(serie2)

ADF Statistic: -3.332425645344991
p-value: 0.013489819379772967
num lags: 2
Critial Values:
   1% : -3.4880216384691867
   5% : -2.8867966864160075
   10% : -2.5802408234244947
Result: The series is stationary


La série temporelle de l'emploi aux Etats-Unis intégrée à l'ordre 1 est stationnaire selon les test KPSS et AFD.

In [94]:
serie1=tocde['United-States']['Actifs']

In [95]:
kpss_test(serie1)
adf_test(serie1)

serie2 = serie1 - serie1.shift()
serie2.dropna(inplace=True)

kpss_test(serie2)
adf_test(serie2)

C:\Users\SURFACE\Anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1850: FutureWarning: The behavior of using nlags=None will change in release 0.13.Currently nlags=None is the same as nlags="legacy", and so a sample-size lag length is used. After the next release, the default will change to be the same as nlags="auto" which uses an automatic lag length selection method. To silence this warning, either use "auto" or "legacy"
  warnings.warn(msg, FutureWarning)
C:\Users\SURFACE\Anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1882: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  warn_msg.format(direction="smaller"), InterpolationWarning


KPSS Statistic: 0.7821858654012013
p-value: 0.01
num lags: 13
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
Result: The series is not stationary
ADF Statistic: -0.5665598786841473

C:\Users\SURFACE\Anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1850: FutureWarning: The behavior of using nlags=None will change in release 0.13.Currently nlags=None is the same as nlags="legacy", and so a sample-size lag length is used. After the next release, the default will change to be the same as nlags="auto" which uses an automatic lag length selection method. To silence this warning, either use "auto" or "legacy"
  warnings.warn(msg, FutureWarning)
C:\Users\SURFACE\Anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1886: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warn_msg.format(direction="greater"), InterpolationWarning



p-value: 0.8783605257395477
num lags: 6
Critial Values:
   1% : -3.489589552580676
   5% : -2.887477210140433
   10% : -2.580604145195395
Result: The series is not stationary
KPSS Statistic: 0.17304322120821797
p-value: 0.1
num lags: 13
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
Result: The series is stationary
ADF Statistic: -2.6807845966594215
p-value: 0.07742280406617864
num lags: 5
Critial Values:
   1% : -3.489589552580676
   5% : -2.887477210140433
   10% : -2.580604145195395
Result: The series is not stationary


La série temporelle des actifs aux Etats-Unis est stationnaire intégrée à l'ordre 1 selon les tests KPSS et ADF.

In [96]:
serie1=tocde['United-States']['Formation']

In [97]:
kpss_test(serie1)
adf_test(serie1)

serie2 = serie1 - serie1.shift()
serie2.dropna(inplace=True)

kpss_test(serie2)
adf_test(serie2)

KPSS Statistic: 0.8792935265198131
p-value: 0.01
num lags: 13
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
Result: The series is not stationary
ADF Statistic: -0.6611884906481112
p-value: 0.856548153048097
num lags: 1
Critial Values:
   1% : -3.4870216863700767
   5% : -2.8863625166643136
   10% : -2.580009026141913
Result: The series is not stationary
KPSS Statistic: 0.07240057002216738
p-value: 0.1
num lags: 13
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
Result: The series is stationary


C:\Users\SURFACE\Anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1850: FutureWarning: The behavior of using nlags=None will change in release 0.13.Currently nlags=None is the same as nlags="legacy", and so a sample-size lag length is used. After the next release, the default will change to be the same as nlags="auto" which uses an automatic lag length selection method. To silence this warning, either use "auto" or "legacy"
  warnings.warn(msg, FutureWarning)
C:\Users\SURFACE\Anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1882: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  warn_msg.format(direction="smaller"), InterpolationWarning
C:\Users\SURFACE\Anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1850: FutureWarning: The behavior of using nlags=None will change in release 0.13.Currently nlags=None is the same as nlags="legacy", and so a sam

ADF Statistic: -5.31759978372413
p-value: 5.034243261742464e-06
num lags: 0
Critial Values:
   1% : -3.4870216863700767
   5% : -2.8863625166643136
   10% : -2.580009026141913
Result: The series is stationary


La série temporelle de la formation aux Etats-Unis est stationnaire intégrée à l'ordre 1 selon les tests KPSS et AFD.

In [98]:
serie1=tocde['United-States']['Conso']

kpss_test(serie1)
adf_test(serie1)

serie2 = serie1 - serie1.shift()
serie2.dropna(inplace=True)

kpss_test(serie2)
adf_test(serie2)

C:\Users\SURFACE\Anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1850: FutureWarning: The behavior of using nlags=None will change in release 0.13.Currently nlags=None is the same as nlags="legacy", and so a sample-size lag length is used. After the next release, the default will change to be the same as nlags="auto" which uses an automatic lag length selection method. To silence this warning, either use "auto" or "legacy"
  warnings.warn(msg, FutureWarning)


KPSS Statistic: 0.9562502354679252
p-value: 0.01
num lags: 13
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
Result: The series is not stationary
ADF Statistic: -0.5539760704231071
p-value: 0.8810461846151003
num lags: 4
Critial Values:
   1% : -3.4885349695076844
   5% : -2.887019521656941
   10% : -2.5803597920604915
Result: The series is not stationary
KPSS Statistic: 0.10492181261084312
p-value: 0.1
num lags: 13
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
Result: The series is stationary
ADF Statistic: -3.576246262191765
p-value: 0.006231727614894642
num lags: 3
Critial Values:
   1% : -3.4885349695076844
   5% : -2.887019521656941
   10% : -2.5803597920604915
Result: The series is stationary


C:\Users\SURFACE\Anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1882: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  warn_msg.format(direction="smaller"), InterpolationWarning
C:\Users\SURFACE\Anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1850: FutureWarning: The behavior of using nlags=None will change in release 0.13.Currently nlags=None is the same as nlags="legacy", and so a sample-size lag length is used. After the next release, the default will change to be the same as nlags="auto" which uses an automatic lag length selection method. To silence this warning, either use "auto" or "legacy"
  warnings.warn(msg, FutureWarning)
C:\Users\SURFACE\Anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1886: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater t

In [99]:
serie1=tocde['United-States']['Exports']

kpss_test(serie1)
adf_test(serie1)

serie2 = serie1 - serie1.shift()
serie2.dropna(inplace=True)

kpss_test(serie2)
adf_test(serie2)

KPSS Statistic: 0.9535354665350331
p-value: 0.01
num lags: 13
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
Result: The series is not stationary
ADF Statistic: -0.41535905120204186
p-value: 0.9075026091117948
num lags: 1
Critial Values:
   1% : -3.4870216863700767
   5% : -2.8863625166643136
   10% : -2.580009026141913
Result: The series is not stationary
KPSS Statistic: 0.07461938064884595
p-value: 0.1
num lags: 13
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
Result: The series is stationary
ADF Statistic: -6.404710137541772

C:\Users\SURFACE\Anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1850: FutureWarning: The behavior of using nlags=None will change in release 0.13.Currently nlags=None is the same as nlags="legacy", and so a sample-size lag length is used. After the next release, the default will change to be the same as nlags="auto" which uses an automatic lag length selection method. To silence this warning, either use "auto" or "legacy"
  warnings.warn(msg, FutureWarning)
C:\Users\SURFACE\Anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1882: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  warn_msg.format(direction="smaller"), InterpolationWarning
C:\Users\SURFACE\Anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1850: FutureWarning: The behavior of using nlags=None will change in release 0.13.Currently nlags=None is the same as nlags="legacy", and so a sam


p-value: 1.9561891822673754e-08
num lags: 2
Critial Values:
   1% : -3.4880216384691867
   5% : -2.8867966864160075
   10% : -2.5802408234244947
Result: The series is stationary


Les séries des exportations et de la consommation aux Etat-Unis sont stationnaires d'ordre 1. 

In [100]:
serie1=tocde['France']['Exports']

kpss_test(serie1)
adf_test(serie1)

serie2 = serie1 - serie1.shift()
serie2.dropna(inplace=True)

kpss_test(serie2)
adf_test(serie2)

KPSS Statistic: 0.9463216189808877
p-value: 0.01
num lags: 13
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
Result: The series is not stationary
ADF Statistic: -0.596628095004256
p-value: 0.8717421203610168
num lags: 4
Critial Values:
   1% : -3.4885349695076844
   5% : -2.887019521656941
   10% : -2.5803597920604915
Result: The series is not stationary
KPSS Statistic: 0.0691047423630225
p-value: 0.1
num lags: 13
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
Result: The series is stationary
ADF Statistic: -5.7762344942919235
p-value: 5.251605272992117e-07
num lags: 3
Critial Values:
   1% : -3.4885349695076844
   5% : -2.887019521656941
   10% : -2.5803597920604915
Result: The series is stationary


C:\Users\SURFACE\Anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1850: FutureWarning: The behavior of using nlags=None will change in release 0.13.Currently nlags=None is the same as nlags="legacy", and so a sample-size lag length is used. After the next release, the default will change to be the same as nlags="auto" which uses an automatic lag length selection method. To silence this warning, either use "auto" or "legacy"
  warnings.warn(msg, FutureWarning)
C:\Users\SURFACE\Anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1882: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  warn_msg.format(direction="smaller"), InterpolationWarning
C:\Users\SURFACE\Anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1850: FutureWarning: The behavior of using nlags=None will change in release 0.13.Currently nlags=None is the same as nlags="legacy", and so a sam

In [101]:
serie1=tocde['France']['PIB']

kpss_test(serie1)
adf_test(serie1)

serie2 = serie1 - serie1.shift()
serie2.dropna(inplace=True)

kpss_test(serie2)
adf_test(serie2)

KPSS Statistic: 0.9463203939048611
p-value: 0.01
num lags: 13
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
Result: The series is not stationary
ADF Statistic: -0.7408050180956267
p-value: 0.8358951808506699
num lags: 3
Critial Values:
   1% : -3.4880216384691867
   5% : -2.8867966864160075
   10% : -2.5802408234244947
Result: The series is not stationary
KPSS Statistic: 0.10384215499048713
p-value: 0.1
num lags: 13
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
Result: The series is stationary
ADF Statistic: -4.590291487315106
p-value: 0.00013459780302248328
num lags: 2
Critial Values:
   1% : -3.4880216384691867
   5% : -2.8867966864160075
   10% : -2.5802408234244947
Result: The series is stationary


C:\Users\SURFACE\Anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1850: FutureWarning: The behavior of using nlags=None will change in release 0.13.Currently nlags=None is the same as nlags="legacy", and so a sample-size lag length is used. After the next release, the default will change to be the same as nlags="auto" which uses an automatic lag length selection method. To silence this warning, either use "auto" or "legacy"
  warnings.warn(msg, FutureWarning)
C:\Users\SURFACE\Anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1882: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  warn_msg.format(direction="smaller"), InterpolationWarning
C:\Users\SURFACE\Anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1850: FutureWarning: The behavior of using nlags=None will change in release 0.13.Currently nlags=None is the same as nlags="legacy", and so a sam

In [102]:
serie1 = tocde['France']['Chomage'].dropna()
serie1

date
2003-01-01    7.93574
2003-04-01    8.09662
2003-07-01    8.01927
2003-10-01    8.34581
2004-01-01    8.52777
               ...   
2018-10-01    8.76056
2019-01-01    8.75797
2019-04-01    8.45035
2019-07-01    8.43242
2019-10-01    8.13304
Name: Chomage, Length: 68, dtype: object

In [103]:
kpss_test(serie1)
adf_test(serie1)

serie2 = serie1 - serie1.shift()
serie2.dropna(inplace=True)

kpss_test(serie2)
adf_test(serie2)

KPSS Statistic: 0.36416619219496654
p-value: 0.09260077922630752
num lags: 11
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
Result: The series is stationary
ADF Statistic: -1.8443123016565899
p-value: 0.3586941930018869
num lags: 2
Critial Values:
   1% : -3.5352168748293127
   5% : -2.9071540828402367
   10% : -2.5911025443786984
Result: The series is not stationary
KPSS Statistic: 0.21697730844640392
p-value: 0.1
num lags: 11
Critial Values:
   10% : 0.347
   5% : 0.463
   2.5% : 0.574
   1% : 0.739
Result: The series is stationary
ADF Statistic: -3.543978252137125
p-value: 0.006926916565178599
num lags: 1
Critial Values:
   1% : -3.5352168748293127
   5% : -2.9071540828402367
   10% : -2.5911025443786984
Result: The series is stationary


C:\Users\SURFACE\Anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1850: FutureWarning: The behavior of using nlags=None will change in release 0.13.Currently nlags=None is the same as nlags="legacy", and so a sample-size lag length is used. After the next release, the default will change to be the same as nlags="auto" which uses an automatic lag length selection method. To silence this warning, either use "auto" or "legacy"
  warnings.warn(msg, FutureWarning)
C:\Users\SURFACE\Anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:1850: FutureWarning: The behavior of using nlags=None will change in release 0.13.Currently nlags=None is the same as nlags="legacy", and so a sample-size lag length is used. After the next release, the default will change to be the same as nlags="auto" which uses an automatic lag length selection method. To silence this warning, either use "auto" or "legacy"
  warnings.warn(msg, FutureWarning)
C:\Users\SURFACE\Anaconda3\lib\site-packages\statsmodels

Nous avons effectué un certain nombre de tests sur les séries. Parfois, les séries n'était stationnaires qu'à l'ordre 1. Dans ce cas la quelle série devons nous utiliser pour la méthode de contrôle synthétique? La série stationnarisée ou la série originale? 